In [132]:
from IPython.display import display
from IPython.core.display import HTML 
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)
 
import sqlite3
import pandas as pd
import numpy as np
from ipywidgets import *

import os
from os import listdir
from os.path import isfile, join


import plotly
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode()
pd.set_option('mode.chained_assignment',None)

Toggle code

In [51]:
con = sqlite3.connect('ASX.db')
cur = con.cursor()

In [52]:
def add_atr_to_dataframe(dataframe, periods):
    dataframe['TR1'] = abs (dataframe['high'] - dataframe['low'])
    dataframe['TR2'] = abs (dataframe['high'] - dataframe['close'].shift())
    dataframe['TR3'] = abs (dataframe['low'] - dataframe['close'].shift())
    dataframe['TrueRange'] = dataframe[['TR1', 'TR2', 'TR3']].max(axis=1)
    dataframe['ATR'] = df["TrueRange"].rolling(periods).mean()
    

In [53]:
def HHV(series, days):
    '''returns the highest value for the series in the last n days'''
    hhv = series.rolling(window=days,center=False).max().shift()
    return hhv

In [54]:
def LLV(series, days):
    '''returns the lowest value for the series in the last n days'''
    llv = series.rolling(window=days,center=False).min().shift()
    return llv

In [ ]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days + 1)):
        yield start_date + datetime.timedelta(n)

In [78]:
def get_stock_from_db(con, stock, start_date, end_date):
    stock = stock.upper()
    query = '''SELECT *
    FROM stocks
    WHERE Symbol = '%s'
    AND (substr(date, 1, 4) || '-' || substr(date, 5, 2) || '-' || substr(date, 7)) BETWEEN '%s' AND '%s'
    ''' % (stock, start_date, end_date)
    df = pd.read_sql_query(query, con)
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
    
    return df

In [253]:
def add_indicators(df, atr_days, highest_days, lowest_days, sma1_days, sma2_days, sma3_days):
    
    add_atr_to_dataframe(df, atr_days)
    
    #Set the number of days for the highest high (top of the Donchian channel)
    df['HHV'] = HHV(df['close'], highest_days)
    
    #Set the number of days for the lowest low (bottom of the Donchian channel)
    df['LLV'] = LLV(df['close'], lowest_days)
    
    #set the moving averages
    df['SMA1'] = df["close"].rolling(sma1_days).mean()
    df['SMA2'] = df["close"].rolling(sma2_days).mean()
    df['SMA3'] = df["close"].rolling(sma3_days).mean()
    
    #add daily rate of change
    df['daily_roc'] = df['close'].pct_change()

    #add weekly rate of change
    df['weekly_roc'] = df['close'].pct_change(periods=5)
    
    #add daily change in MAs
    df['SMA1_daily_roc'] = df['SMA1'].pct_change()
    df['SMA2_daily_roc'] = df['SMA2'].pct_change()
    df['SMA3_daily_roc'] = df['SMA3'].pct_change()
    
    #add weekly change in MAs
    df['SMA1_weekly_roc'] = df['SMA1'].pct_change(periods=5)
    df['SMA2_weekly_roc'] = df['SMA2'].pct_change(periods=5)
    df['SMA3_weekly_roc'] = df['SMA3'].pct_change(periods=5)

In [280]:
def add_trade_triggers(df):
    '''code your entry and exit conditions in this function'''
    #entry conditions for a long trade
    df['long_entry'] = (df['close'] > df['HHV']) & (df['SMA3_weekly_roc'] > 0)
    
    #exit conditions for a long trade
    df['long_exit'] = (df['close'] < df['SMA3'])

In [281]:
def add_trade_state(df):

    in_trade = False
    trade_states = []
    for index, row in df.iterrows():
        if row['long_entry'] and not row['long_exit']:
            in_trade = True

        trade_states.append(in_trade)

        if in_trade and row['long_exit']:
            in_trade = False
        
    df['in_trade'] = trade_states

In [397]:
def create_trade_db(df):
    
    entries = df[df['long_entry'].shift() == True][['symbol', 'date', 'open']]
    exits = df[(df['long_exit'].shift() == True) & (df['in_trade'].shift() == True)][['symbol', 'date', 'open']]
    trades = entries
    
    exit_dates = []
    exit_prices = []

    for index, row in entries.iterrows():
        try:
            exit_dates.append(exits[exits['date'] > row['date']]['date'].iloc[0])
        except:
            exit_dates.append('NaN')

        try:
            exit_prices.append(exits[exits['date'] > row['date']]['open'].iloc[0])
        except:
            exit_prices.append('NaN')
            
    trades.rename(columns={'symbol': 'Symbol', 'date': 'Entry_Date', 'open': 'Entry_Price'},inplace=True)
    trades['stop_loss'] = ''
    trades['initial_risk'] = ''
    trades['Exit_Date'] = exit_dates
    trades['Exit_Price'] = exit_prices
    trades.dropna(inplace=True)
    
    return trades

In [350]:
#set the start and end dates 
start_date = '2010-01-01'
end_date = '2016-12-31'

In [384]:
highest_days = 65
lowest_days = 20
sma1_days = 20
sma2_days = 50
sma3_days = 150
atr_days = 15

In [402]:
stocks = ['BHP', 'ASX', 'CBA', 'anz']

In [403]:
trades = pd.DataFrame()

In [404]:
for stock in stocks:
    df = get_stock_from_db(con, stock, start_date, end_date)
    add_indicators(df, atr_days, highest_days, lowest_days, sma1_days, sma2_days, sma3_days)
    add_trade_triggers(df)
    add_trade_state(df)
    #print df.head()
    temp_df = create_trade_db(df)
    #print temp_df.head()
    trades = trades.append(temp_df)

In [405]:
trades.head()

,Symbol,Entry_Date,Entry_Price,stop_loss,initial_risk,Exit_Date,Exit_Price
239,BHP,2010-12-10,45.30,,,2011-03-16,43.52
242,BHP,2010-12-15,45.48,,,2011-03-16,43.52
248,BHP,2010-12-23,45.96,,,2011-03-16,43.52
249,BHP,2010-12-24,46.17,,,2011-03-16,43.52
276,BHP,2011-02-07,46.44,,,2011-03-16,43.52


In [406]:
trades.to_csv(path_or_buf=join('C:\Users\Aaron\Documents\Trading', 'trade_db.csv'), index=False, mode='w')